##  Optimze and a Dense Neural Network for gap filling and feature identification

** With a few tweaks to RepRunner, an LSTM can be run instead

In [39]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
# from functools import partial
from multiprocessing import Pool
from functools import partial
from sklearn import metrics
# from sklearn.model_selection import train_test_split

## Personal Modules
import ReadStandardTimeFill as RSTF
import importlib
import DenseNet as Dense
import MiscFuncs as MF
importlib.reload(Dense)
importlib.reload(RSTF)
importlib.reload(MF)

# %matplotlib inline

%matplotlib notebook
%config IPCompleter.greedy=True

from scipy.optimize import minimize, curve_fit
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.externals import joblib
from matplotlib import cm

from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

from ipywidgets import FloatProgress
from IPython.display import display
import os  
import shutil
from keras import backend as K
try:pool.close()
except:pass


def Test(params,X,y,YScaled,XScaled,pool):
    return(np.random.rand(params['K']))


def ModSelect(Scope,Site):
    if Site == 'Illisarvik':
        if Scope == 'Full':
            Model = ['wind_speed','air_pressure','PPFD_Avg','AirTC_Avg','VPD',
                    'Temp','VWC','Sedge','Shrub','Grass','Sparse','Out_of_Basin']
        if Scope == 'Test':
            Model = ['PPFD_Avg','Sedge']
    if Site == 'FishIsland':
        BaseFactors = []
        if Scope == 'Full':
            Model = ['H','Wind Spd','air pressure','Ta','Rn','PPFD','Rain','Water Table',
            'Ts 2.5 cm','Ts 15 cm','VWC','Active Layer','24H Rain','Wtr Tbl Trnd']
        if Scope == 'Test':
            Model = ['H','Water Table','Wind Spd','Active Layer']
    return(Model)

def Combos(Model,L,factor=None):
    Models=[]
    for c in combinations(Model,L):
        c = list(c)
        if factor is None:
            Models.append(c)
        else:
            for f in factor:
                f = f.split('+')
                if set(f).issubset(set(c)) and c not in Models:
                    Models.append(c)
                    
    print('Models: ',Models)
    return(Models)

def Stats(mse,j,i,params):
    df = pd.DataFrame(index = [str(j)+'_'+str(i)],
                      data={'Model':[params['Model']],
                            'MSE':[mse.mean()],
                            'Size':j,
                            'Number':i,
                            'SE':[mse.std()/params['K']**.5],
                            'Performance':0})
    return(df)

def t(p,n):
    alpha = 1-p
    df = n-1
    return(stats.t.ppf(alpha,df))


FirstRun = True
# FirstRun = False

MP=True
processes=3
Scope = 'Full'
cwd = os.getcwd()
# for Site in ['Illisarvik','FishIsland']:
Site='Illisarvik'
target='fco2'
T= 1#t(0.05,params['K'])


# Model

In [40]:

if __name__ == '__main__' and FirstRun == True:
    XVarriables=ModSelect(Scope,Site)
    prog = FloatProgress(min=0, max=len(XVarriables)+1,description='Running:') # instantiate the bar
    display(prog) # display the bar
    try:shutil.rmtree(cwd+'/'+Site+'/'+target+'/')
    except:pass
    os.mkdir(cwd+'/'+Site+'/'+target+'/')
    for j in range(1,len(XVarriables)+1):
        if j == 1: Inputs = (Combos(XVarriables,j))
        else: Inputs = (Combos(XVarriables,j,Factors))
        i = 0
        for Input in Inputs:
            params = Dense.Params(Scope,target,MP)
            params['Dpath'] = cwd+'/'+Site+'/'
            params['Spath'] = params['Dpath']+'/'+target+'/'+str(j)+'_'+str(i)+'/'
            try:os.mkdir(params['Spath'])
            except:pass
            params['Sname'] = 'Y_'
            params['Inputs'] = Input
            params['Model'] = '+'.join(params['Inputs'])
            RST = RSTF.ReadStandardTimeFill(params,'ECData.csv')#,resample='2H')
            RST.Scale(params['target'],params['Inputs']) 
            y = RST.y*1.0
            X = RST.X*1.0
            params['N']=int(y.shape[0]/30)
            params['Memory'] = (math.floor(100/params['proc'])- 5/params['proc']) * .01
            Y_hat=[]
            y_true=[]
            X_true=[]
            index=[]
            ones=[]
            if MP == False:
                for k in range(params['K']):
                    results = Dense.TTV_Split(k,params,X,y)
                    Y_hat.append(RST.YScaled.inverse_transform(results[0]))
                    y_true.append(RST.YScaled.inverse_transform(results[1]))
                    X_true.append(RST.XScaled.inverse_transform(results[2]))
                    index.append(results[3])
                    ones.append(results[4])
            else:
                pool = Pool(processes=processes,maxtasksperchild=75)
                for k,results in enumerate(pool.imap(partial(Dense.TTV_Split,params=params,X=X,y=y),range(params['K']))):
                    Y_hat.append(RST.YScaled.inverse_transform(results[0]))
                    y_true.append(RST.YScaled.inverse_transform(results[1]))
                    X_true.append(RST.XScaled.inverse_transform(results[2]))
                    index.append(results[3])
                    ones.append(results[4])
                pool.close()
            Y_hat = np.squeeze(np.asanyarray(Y_hat))
            y_true = np.squeeze(np.asanyarray(y_true))
            X_true = np.asanyarray(X_true)
            index = np.asanyarray(index)
            ones = np.asanyarray(ones)
            params['Memory'] = .95
            if MP == False:
                for k in range(1):
                     mse = Dense.Sort_outputs(k,params,Y_hat,y_true,X_true,index,ones)
            else:
                pool = Pool(processes=1,maxtasksperchild=75)
                for k,results in enumerate(pool.imap(partial(Dense.Sort_outputs,params=params,
                 Y_hat=Y_hat,y_true=y_true,X_true=X_true,index=index,ones=ones),range(1))):
                     mse = results
                pool.close()
            if i == 0:Level = Stats(mse,j,i,params)
            else:Level = Level.append(Stats(mse,j,i,params))
            i += 1
            prog.value=j+i/len(Inputs)
        Min = Level.loc[Level['MSE']==Level['MSE'].min()]
        T= 1#t(0.05,params['K'])
        Factors = Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Model'].values
        print(Factors)
        Level.loc[Level['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Performance']=1
        if j == 1:Records = Level
        else:Records = Records.append(Level)

    Records = Records.reset_index(drop=True)
    Min = Records.loc[Records['MSE']==Records['MSE'].min()]
    Records.loc[Records['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Performance']=2
    Records.loc[Records['MSE']==Min['MSE'].values[0],'Performance']=3
    Honorable = Records.loc[Records['Performance']==1]
    Top = Records.loc[Records['Performance']==2]
    Best = Records.loc[Records['Performance']==3]
    plt.figure()
    plt.bar(Honorable.index,Honorable['MSE'],yerr=Honorable['SE'],color='blue')
    plt.bar(Top.index,Top['MSE'],yerr=Top['SE'],color='green')
    plt.bar(Best.index,Best['MSE'],yerr=Best['SE'],color='red')
    Records.drop(Records.columns[Records.columns.str.contains('Unnamed',case = False)],axis = 1)
    Records.to_csv(params['Dpath']+'/'+target+'_Runs.csv')
    FirstRun = False
else:    
    print('OldRun!')
#     params['Spath'] = params['Dpath']+'/'+target+'/'+str(j)+'_'+str(i)+'/'
    params = Dense.Params(Scope,target,MP)
    params['Dpath'] = cwd+'/'+Site+'/'
    RST = RSTF.ReadStandardTimeFill(params,'ECData.csv')#,resample='2H')
    params = Dense.Params(Scope,target,MP)
    params['Dpath'] = cwd+'/'+Site+'/'
    Records = pd.read_csv(params['Dpath']+'/'+target+'_Runs.csv',delimiter = ',',header = 0)
    Min = Records.loc[Records['MSE']==Records['MSE'].min()]
    Records.loc[Records['MSE']<=Min['MSE'].values[0]+Min['SE'].values[0]*T,'Performance']=2
    Records.loc[Records['MSE']==Min['MSE'].values[0],'Performance']=3
    T= 1#t(0.05,params['K'])
    Honorable = Records.loc[Records['Performance']==1]
    Top = Records.loc[Records['Performance']==2]
    Best = Records.loc[Records['Performance']==3]
    plt.figure()
    plt.bar(Honorable.index,Honorable['MSE'],yerr=Honorable['SE'],color='blue')
    plt.bar(Top.index,Top['MSE'],yerr=Top['SE'],color='green')
    plt.bar(Best.index,Best['MSE'],yerr=Best['SE'],color='red')
    Records.to_csv(params['Dpath']+'/'+target+'_Runs.csv')
    print(Records)

A Jupyter Widget

Models:  [['wind_speed'], ['air_pressure'], ['PPFD_Avg'], ['AirTC_Avg'], ['VPD'], ['Temp'], ['VWC'], ['Sedge'], ['Shrub'], ['Grass'], ['Sparse'], ['Out_of_Basin']]
['PPFD_Avg']
Models:  [['wind_speed', 'PPFD_Avg'], ['air_pressure', 'PPFD_Avg'], ['PPFD_Avg', 'AirTC_Avg'], ['PPFD_Avg', 'VPD'], ['PPFD_Avg', 'Temp'], ['PPFD_Avg', 'VWC'], ['PPFD_Avg', 'Sedge'], ['PPFD_Avg', 'Shrub'], ['PPFD_Avg', 'Grass'], ['PPFD_Avg', 'Sparse'], ['PPFD_Avg', 'Out_of_Basin']]
['air_pressure+PPFD_Avg' 'PPFD_Avg+VPD']
Models:  [['wind_speed', 'air_pressure', 'PPFD_Avg'], ['wind_speed', 'PPFD_Avg', 'VPD'], ['air_pressure', 'PPFD_Avg', 'AirTC_Avg'], ['air_pressure', 'PPFD_Avg', 'VPD'], ['air_pressure', 'PPFD_Avg', 'Temp'], ['air_pressure', 'PPFD_Avg', 'VWC'], ['air_pressure', 'PPFD_Avg', 'Sedge'], ['air_pressure', 'PPFD_Avg', 'Shrub'], ['air_pressure', 'PPFD_Avg', 'Grass'], ['air_pressure', 'PPFD_Avg', 'Sparse'], ['air_pressure', 'PPFD_Avg', 'Out_of_Basin'], ['PPFD_Avg', 'AirTC_Avg', 'VPD'], ['PPFD_Avg', 'VPD',

['wind_speed+air_pressure+PPFD_Avg+AirTC_Avg+VPD+Temp+VWC+Shrub'
 'wind_speed+air_pressure+PPFD_Avg+AirTC_Avg+VPD+Temp+VWC+Out_of_Basin'
 'wind_speed+air_pressure+PPFD_Avg+VPD+Temp+VWC+Shrub+Out_of_Basin'
 'wind_speed+air_pressure+PPFD_Avg+VPD+Temp+VWC+Sparse+Out_of_Basin']
Models:  [['wind_speed', 'air_pressure', 'PPFD_Avg', 'AirTC_Avg', 'VPD', 'Temp', 'VWC', 'Sedge', 'Shrub'], ['wind_speed', 'air_pressure', 'PPFD_Avg', 'AirTC_Avg', 'VPD', 'Temp', 'VWC', 'Sedge', 'Out_of_Basin'], ['wind_speed', 'air_pressure', 'PPFD_Avg', 'AirTC_Avg', 'VPD', 'Temp', 'VWC', 'Shrub', 'Grass'], ['wind_speed', 'air_pressure', 'PPFD_Avg', 'AirTC_Avg', 'VPD', 'Temp', 'VWC', 'Shrub', 'Sparse'], ['wind_speed', 'air_pressure', 'PPFD_Avg', 'AirTC_Avg', 'VPD', 'Temp', 'VWC', 'Shrub', 'Out_of_Basin'], ['wind_speed', 'air_pressure', 'PPFD_Avg', 'AirTC_Avg', 'VPD', 'Temp', 'VWC', 'Grass', 'Out_of_Basin'], ['wind_speed', 'air_pressure', 'PPFD_Avg', 'AirTC_Avg', 'VPD', 'Temp', 'VWC', 'Sparse', 'Out_of_Basin'], ['wind

<IPython.core.display.Javascript object>

# Map Results

In [41]:
alt_mod=[False,'PPFD_Avg+wind_speed+Sedge']
BestMod = Best['Model'].values[0]
Project = False
Fill = False
if alt_mod[0] == True:
    Model = alt_mod[1]
else:
    Model = BestMod
idx = Records['index'].loc[Records['Model']==Model].values[0]
Model = Model.split('+')
params['Spath']=(params['Dpath']+'/'+target+'/'+idx+'/')
params['Inputs']=Model
RST.Scale(params['target'],params['Inputs'],ScalePath=params['Spath'])

KeyError: 'index'

# Predict

In [ ]:

# print(Data[params['target']])

for ip in params['Inputs']:
    RST.Data[ip] = RST.Data[ip].mean()

Key = 'Sedge'
KeyRange = {'min':0,'max':1}
RST.Data[Key] = np.linspace(KeyRange['min'],KeyRange['max'],RST.Data[Key].shape[0])
RST.Scale(params['target'],params['Inputs'],ScalePath=params['Spath'],Project=Project)
    
# print(Data[params['target']])

if __name__ == '__main__':
    if Fill == True:
        X = RST.X_fill
    else:
        X = RST.X
    params['Sname']='Y_'
    Y_fill = []
    MSE = []
#     for i in range(params['K']):
        
        
        
    if MP == False:
        for k in range(params['K']):
            Y = Dense.Load_Model(k,X,params)
            Y = RST.YScaled.inverse_transform(Y)
            if Fill == False:
                mse = (metrics.mean_squared_error(RST.y,Y))
                MSE.append(mse)
            Y_fill.append(Y)
    else:
        pool = Pool(processes=1,maxtasksperchild=75)
        for k,results in enumerate(pool.imap(partial(Dense.Load_Model,X=X,params=params),range(params['K']))):
            Y = results
            Yold = Y+0
            Y = RST.YScaled.inverse_transform(Y)
#             plt.figure()
#             plt.scatter(Y,Yold)
#             plt.xlabel('Sca;e')
#             print(RST.YScaled)
            if Fill == False:
                mse = (metrics.mean_squared_error(RST.y,Y))
                MSE.append(mse)
            Y_fill.append(Y)                       
        pool.close()
#         params['iteration']=i

#         Y = Dense.Load_Model(i,X,params)
#         Model = Dense.Load_Weights(Empty_Mod,params) 
#         Y = RST.YScaled.inverse_transform(Y)#Model.predict(X).reshape(-1,1))
#         if Fill == False:
#             mse = (metrics.mean_squared_error(RST.y,Y))
#             MSE.append(mse)
#         Y_fill.append(Y)
    Y_fill = np.asanyarray(Y_fill).mean(axis=-1)
    Y_fill_bar = Y_fill.mean(axis=0)
    if Fill == False:
        MSE = np.asanyarray(MSE)
        CI = stats.t.ppf(1-0.025,k)*MSE.std()/(k)**.5
        print(CI)

    YStandard = joblib.load(params['Spath']+"YVar_scaler.save") 
    params['Sname']='Var'
#     params['iteration']=1
    params['Loss']='Boot_Loss'
#     YVar = Dense.Load_Model(1,X,params)
#     Model = Dense.Load_Weights(Empty_Mod,params)        
    if MP == False:
        for k in range(1,2):
            YVar = Dense.Load_Model(k,X,params)
            YVar = RST.YScaled.inverse_transform(YVar)
#             if Fill == False:
#                 mse = (metrics.mean_squared_error(RST.y,Y))
#                 MSE.append(mse)
#             Y_fill.append(Y)
    else:
        pool = Pool(processes=1,maxtasksperchild=75)
        for k,results in enumerate(pool.imap(partial(Dense.Load_Model,X=X,params=params),range(1,2))):
            YVar = results
            YVar = RST.YScaled.inverse_transform(YVar)
#             if Fill == False:
#                 mse = (metrics.mean_squared_error(RST.y,Y))
#                 MSE.append(mse)
#             Y_fill.append(Y)                       
        pool.close()
#     YVar=YStandard.inverse_transform(YVar)#Model.predict(X).reshape(-1,1))
    X_back = np.squeeze(RST.XScaled.inverse_transform(X))


Data = pd.DataFrame(data=X_back,columns=params['Inputs'])
Data[target] = np.squeeze(Y_fill_bar)
Data['True'] = RST.Ytru#.YScaled.inverse_transform(RST.y)
Data['SE'] = 1/(params['K']-1)*((Y_fill-Y_fill_bar)**2).sum(axis=0)
Data['Var'] = np.squeeze(YVar)
Data['CI']=stats.t.ppf(1-0.025,params['K'])*(Data['SE'])**.5
Data['PI']=stats.t.ppf(1-0.025,params['K'])*((Data['Var']+Data['SE'])**.5) #the accuracy of our estimate with respect to the observed output

# print(Data['CI'].mean())
print(params['Inputs'])
print('Prediction mean: ',Data[target].mean())
print('Target mean: ',Data['True'].mean())
Data['Fill'] = Data['True'].fillna(Data[target])
print('GapFilled mean: ',Data['Fill'].mean())

Data.loc[np.isnan(Data['PI'])==True,'PI']=Data['CI']

In [ ]:

plt.figure(figsize=(8,7))
Data = Data.sort_values(by=Key)
# Data.index = Data[Key]

Data = Data[np.isfinite(Data['True'])]

plt.scatter(RST.Master[Key],RST.Master[target],edgecolor='black',facecolor='white')
plt.plot(Data[Key],Data[target],
         label= params['target']+' Model\nr^2: '+str(np.round(metrics.r2_score(Data['True'],
                                                                   Data[params['target']])**2,3)))
# plt.plot(Data.index,Data['Var'],label= params['target']+
# ' Model\nRMSE: '+str(np.round(metrics.mean_squared_error(Data['True'],
#                                                                    Data[params['target']])**2,3)))


plt.fill_between(Data[Key], Data[target]-Data['PI'], 
                 Data[target]+Data['PI'],  color = 'green', alpha = 0.4, 
                 label = '95% PI')
plt.fill_between(Data[Key], Data[target]-Data['CI'], 
                 Data[target]+Data['CI'],  color = 'red', alpha = 0.4, 
                 label = '95% CI')
plt.legend()
# print(RST.Master)

plt.figure()

# plt.plot(Data[Key], Data['Var'])

plt.scatter(Data[target],Data['True'])

# print(Data['SE'])

NameError: name 'np' is not defined